In [7]:
from collections import defaultdict
from collections import defaultdict, Counter
import requests
import json
import time
from datetime import datetime, timedelta
from pandas import json_normalize
import pandas as pd
import calendar
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor, as_completed

import asyncio
import aiohttp
import json


mes_passado = datetime.now() - timedelta(days=30)
mes_passado_inicio = mes_passado.strftime('%Y-%m-01 00:00:00')
mes_passado_fim = mes_passado.strftime('%Y-%m-31 23:59:59')

from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

token = os.getenv('TOKEN_API')

BASE_URL = 'http://10.0.100.128:5009'

In [ ]:
# URLs base
URL_RADUSUARIOS = "https://assinante.nmultifibra.com.br/webservice/v1/radusuarios"
URL_CONTRATO = "https://assinante.nmultifibra.com.br/webservice/v1/cliente_contrato"

# IDs dos planos que queremos buscar
PLANOS_IDS = [4, 60]

# Headers se necessário (ex: token de autenticação)
headers = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

# Lista final dos clientes ativos
clientes_ativos = []

for plano_id in PLANOS_IDS:
    payload_rad = {
        "qtype": "id_grupo",
        "query": str(plano_id),
        "oper": "=",
        "page": "1",
        "rp": "1000"  # Número de registros por página (ajuste conforme necessidade)
    }
    response_rad = requests.post(URL_RADUSUARIOS, json=payload_rad, headers=headers)
    dados_rad = response_rad.json()

    for usuario in dados_rad.get("registros", []):
        id_contrato = usuario["id_contrato"]

        # Consulta de contrato individual
        payload_contrato = {
            "qtype": "id",
            "query": id_contrato,
            "oper": "=",
            "page": "1",
            "rp": "1"
        }
        response_contrato = requests.post(URL_CONTRATO, json=payload_contrato, headers=headers)
        dados_contrato = response_contrato.json()

        registros = dados_contrato.get("registros", [])
        if registros and registros[0].get("status") == "A":
            cliente_final = {
                "id_cliente": usuario["id_cliente"],
                "id_contrato": id_contrato,
                "login": usuario["login"],
                "ip": usuario["ip"],
                "plano_id": usuario["id_grupo"]
            }
            clientes_ativos.append(cliente_final)

# Resultado final em JSON
import json
print(json.dumps(clientes_ativos, indent=2, ensure_ascii=False))

In [15]:
URL_CONTRATO = "https://assinante.nmultifibra.com.br/webservice/v1/cliente_contrato"

HEADERS = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

contratos_ativos = []

async def buscar_contratos_ativos_por_plano(session, id_vd_contrato, paginas=20):
    contratos = []
    for page in range(1, paginas + 1):
        payload = {
            "qtype": "id_vd_contrato",
            "query": str(id_vd_contrato),
            "oper": "=",
            "page": str(page),
            "rp": "1000"
        }
        async with session.post(URL_CONTRATO, json=payload, headers=HEADERS) as resp:
            dados = await resp.json(content_type=None)
            registros = dados.get("registros", [])
            if not registros:
                break
            ativos = [c for c in registros if c.get("status") == "A"]
            contratos.extend(ativos)
            print(f"Plano {id_vd_contrato} - Página {page} - {len(ativos)} contratos ativos")
            if len(registros) < 1000:
                break
    return contratos

async def main():
    planos = [58, 93, 52, 94, 60, 4]  # IDs dos planos que quer consultar
    todos_contratos = []

    async with aiohttp.ClientSession() as session:
        for plano in planos:
            resultado = await buscar_contratos_ativos_por_plano(session, plano, paginas=20)
            todos_contratos.extend(resultado)

    print(f"\nTotal contratos ativos encontrados em todos os planos: {len(todos_contratos)}")

    with open("contratos_ativos_todos_planos.json", "w", encoding="utf-8") as f:
        json.dump(todos_contratos, f, indent=2, ensure_ascii=False)
        
await main()

Plano 58 - Página 1 - 299 contratos ativos
Plano 58 - Página 2 - 523 contratos ativos
Plano 58 - Página 3 - 524 contratos ativos
Plano 58 - Página 4 - 490 contratos ativos
Plano 58 - Página 5 - 579 contratos ativos
Plano 58 - Página 6 - 357 contratos ativos
Plano 93 - Página 1 - 804 contratos ativos
Plano 93 - Página 2 - 816 contratos ativos
Plano 93 - Página 3 - 827 contratos ativos
Plano 93 - Página 4 - 816 contratos ativos
Plano 93 - Página 5 - 833 contratos ativos
Plano 93 - Página 6 - 825 contratos ativos
Plano 93 - Página 7 - 227 contratos ativos
Plano 52 - Página 1 - 127 contratos ativos
Plano 94 - Página 1 - 65 contratos ativos
Plano 60 - Página 1 - 164 contratos ativos
Plano 60 - Página 2 - 470 contratos ativos
Plano 60 - Página 3 - 469 contratos ativos
Plano 60 - Página 4 - 464 contratos ativos
Plano 60 - Página 5 - 436 contratos ativos
Plano 60 - Página 6 - 456 contratos ativos
Plano 60 - Página 7 - 500 contratos ativos
Plano 4 - Página 1 - 100 contratos ativos
Plano 4 - Pág

In [24]:
URL_CONTRATO = "https://assinante.nmultifibra.com.br/webservice/v1/cliente_contrato"

HEADERS = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

contratos_ativos = []

async def buscar_contratos_ativos_por_plano(session, id_vd_contrato, paginas=20):
    contratos = []
    for page in range(1, paginas + 1):
        payload = {
            "qtype": "id_vd_contrato",
            "query": str(id_vd_contrato),
            "oper": "=",
            "page": str(page),
            "rp": "1000"
        }
        async with session.post(URL_CONTRATO, json=payload, headers=HEADERS) as resp:
            dados = await resp.json(content_type=None)
            registros = dados.get("registros", [])
            if not registros:
                break
            ativos = [c for c in registros if c.get("status") == "A"]
            contratos.extend(ativos)
            print(f"Plano {id_vd_contrato} - Página {page} - {len(ativos)} contratos ativos")
            if len(registros) < 1000:
                break
    return contratos

async def main():
    planos = [58, 93, 52, 94, 60, 4]
    todos_contratos = []

    async with aiohttp.ClientSession() as session:
        for plano in planos:
            resultado = await buscar_contratos_ativos_por_plano(session, plano, paginas=20)
            todos_contratos.extend(resultado)

    # Campos que você quer manter
    campos_filtrados = ["status", "status_internet", "id_cliente", "id_vd_contrato", "contrato"]

    # Filtra cada contrato para conter só esses campos
    contratos_filtrados = [
        {chave: contrato.get(chave, "") for chave in campos_filtrados}
        for contrato in todos_contratos
    ]

    print(f"\nTotal contratos ativos encontrados em todos os planos: {len(contratos_filtrados)}")

    with open("contratos_ativos_todos_planos_filtrado.json", "w", encoding="utf-8") as f:
        json.dump(contratos_filtrados, f, indent=2, ensure_ascii=False)

await main()

Plano 58 - Página 1 - 299 contratos ativos
Plano 58 - Página 2 - 523 contratos ativos
Plano 58 - Página 3 - 524 contratos ativos
Plano 58 - Página 4 - 490 contratos ativos
Plano 58 - Página 5 - 579 contratos ativos
Plano 58 - Página 6 - 357 contratos ativos
Plano 93 - Página 1 - 804 contratos ativos
Plano 93 - Página 2 - 816 contratos ativos
Plano 93 - Página 3 - 827 contratos ativos
Plano 93 - Página 4 - 816 contratos ativos
Plano 93 - Página 5 - 833 contratos ativos
Plano 93 - Página 6 - 825 contratos ativos
Plano 93 - Página 7 - 226 contratos ativos
Plano 52 - Página 1 - 127 contratos ativos
Plano 94 - Página 1 - 65 contratos ativos
Plano 60 - Página 1 - 164 contratos ativos
Plano 60 - Página 2 - 470 contratos ativos
Plano 60 - Página 3 - 469 contratos ativos
Plano 60 - Página 4 - 464 contratos ativos
Plano 60 - Página 5 - 436 contratos ativos
Plano 60 - Página 6 - 456 contratos ativos
Plano 60 - Página 7 - 500 contratos ativos
Plano 4 - Página 1 - 100 contratos ativos
Plano 4 - Pág

In [ ]:
URL_CLIENTE = "https://assinante.nmultifibra.com.br/webservice/v1/cliente"

HEADERS = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

# Carrega o JSON com os contratos ativos
with open("contratos_ativos_todos_planos.json", "r", encoding="utf-8") as f:
    contratos = json.load(f)

# Extrair ids únicos de clientes
ids_clientes = list({contrato["id_cliente"] for contrato in contratos})

campos_desejados = [
    "telefone_comercial",
    "telefone_celular",
    "whatsapp",
    "endereco",
    "bairro",
    "numero",
    "cep",
    "razao",
    "data_cadastro"
]

async def consultar_cliente(session, id_cliente):
    payload = {
        "qtype": "id",
        "query": str(id_cliente),
        "oper": "=",
        "page": "1",
        "rp": "1"
    }
    try:
        async with session.post(URL_CLIENTE, json=payload, headers=HEADERS) as resp:
            if resp.status != 200:
                print(f"Cliente {id_cliente}: status HTTP {resp.status}")
                return None
            dados = await resp.json(content_type=None)
            registros = dados.get("registros", [])
            if registros:
                cliente = registros[0]
                # Retorna só os campos desejados
                return {campo: cliente.get(campo, "") for campo in campos_desejados}
    except Exception as e:
        print(f"Erro ao consultar cliente {id_cliente}: {e}")
    return None

async def main():
    resultados = []
    async with aiohttp.ClientSession() as session:
        tarefas = [consultar_cliente(session, cid) for cid in ids_clientes]
        respostas = await asyncio.gather(*tarefas)
        for r in respostas:
            if r:
                resultados.append(r)

    print(f"Total clientes consultados: {len(resultados)}")

    with open("clientes_resumidos.json", "w", encoding="utf-8") as f:
        json.dump(resultados, f, indent=2, ensure_ascii=False)

await main()

Total clientes consultados: 17449


In [5]:
URL_CLIENTE = "https://assinante.nmultifibra.com.br/webservice/v1/cliente"

HEADERS = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

# Carrega o JSON com os contratos ativos filtrados
with open("contratos_ativos_todos_planos_filtrado.json", "r", encoding="utf-8") as f:
    contratos = json.load(f)

# Mapeia os contratos por id_cliente (caso haja mais de um, pega o primeiro)
contratos_por_cliente = {}
for contrato in contratos:
    contratos_por_cliente[contrato["id_cliente"]] = contrato

# Extrair ids únicos de clientes
ids_clientes = list(contratos_por_cliente.keys())

campos_desejados = [
    "telefone_comercial",
    "telefone_celular",
    "whatsapp",
    "endereco",
    "bairro",
    "numero",
    "cep",
    "razao",
    "data_cadastro"
]

async def consultar_cliente(session, id_cliente):
    payload = {
        "qtype": "id",
        "query": str(id_cliente),
        "oper": "=",
        "page": "1",
        "rp": "1"
    }
    try:
        async with session.post(URL_CLIENTE, json=payload, headers=HEADERS) as resp:
            if resp.status != 200:
                print(f"Cliente {id_cliente}: status HTTP {resp.status}")
                return None
            dados = await resp.json(content_type=None)
            registros = dados.get("registros", [])
            if registros:
                cliente = registros[0]
                dados_cliente = {campo: cliente.get(campo, "") for campo in campos_desejados}

                # Adiciona o contrato correspondente
                contrato = contratos_por_cliente.get(str(id_cliente))
                if contrato:
                    dados_cliente["contrato"] = contrato

                return dados_cliente
    except Exception as e:
        print(f"Erro ao consultar cliente {id_cliente}: {e}")
    return None

async def main():
    resultados = []
    async with aiohttp.ClientSession() as session:
        tarefas = [consultar_cliente(session, cid) for cid in ids_clientes]
        respostas = await asyncio.gather(*tarefas)
        for r in respostas:
            if r:
                resultados.append(r)

    print(f"Total clientes consultados: {len(resultados)}")

    with open("clientes_com_contrato.json", "w", encoding="utf-8") as f:
        json.dump(resultados, f, indent=2, ensure_ascii=False)

await main()

Total clientes consultados: 17449


In [95]:

api_url = "https://assinante.nmultifibra.com.br/webservice/v1"
api_key = token

#<># Funções para facilitar a vida #<>#

# Mapeamento
def mapeamento(rota, id, traducao):
    url = f'{api_url}/{rota}'
    
    payload = json.dumps({
        'qtype': '',
        'query': '',
        'oper': '',
        'page': '1',
        'rp': '999999'
    })
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': api_key,
        'ixcsoft': 'listar'
    }
    
    response = requests.request('POST', url, headers=headers, data=payload)
    
    df_mapping = pd.json_normalize(response.json()['registros'])
    df_mapping[f'{traducao}'] = df_mapping[f'{traducao}'].str.upper()
    df_mapping = df_mapping.set_index(f'{id}')[f'{traducao}'].to_dict()
    
    return df_mapping

# Requisicao IXC
def requisicao_ixc(rota, query='[]'):
    url = f'{api_url}/{rota}'
    
    payload = json.dumps({
        'qtype': '',
        'query': '',
        'oper': '',
        'page': '1',
        'rp': '999999',
        'grid_param': f'{query}'
    })
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': api_key,
        'ixcsoft': 'listar'
    }
    
    response = requests.request('POST', url, headers=headers, data=payload)
    
    df = pd.json_normalize(response.json()['registros'])
    
    return df

# Requisicao IXC entre tabelas
def requisicao_ixc_especifica(rota, ids, tabela, query='[]', max_workers=30, estrategia=None):
    
    inicio = time.time()
    ids_unicos = list(set(ids))
    
    def analisar_tabela():
        url = f'{api_url}/{rota}'
        payload = json.dumps({
            'qtype': tabela,
            'query': '',
            'oper': 'LIKE',
            'page': '1',
            'rp': '1',
            'grid_param': f'{query}'
        })
        
        headers = {
            'Content-Type': 'application/json',
            'Authorization': api_key,
            'ixcsoft': 'listar'
        }
        
        try:
            response = requests.post(url, headers=headers, data=payload, timeout=10)
            if response.status_code == 200:
                data = response.json()
                return int(data.get('total', 0))
        except:
            pass
        return None

    def executar_bulk():
        url = f'{api_url}/{rota}'
        payload = json.dumps({
            'qtype': tabela,
            'query': '',
            'oper': 'LIKE',
            'page': '1',
            'rp': '999999',
            'grid_param': f'{query}'
        })
        
        headers = {
            'Content-Type': 'application/json',
            'Authorization': api_key,
            'ixcsoft': 'listar'
        }
        
        try:
            response = requests.post(url, headers=headers, data=payload, timeout=120)
            
            if response.status_code == 200:
                data = response.json()
                registros = data.get("registros", [])
                
                ids_set = set(map(str, ids_unicos))
                registros_filtrados = [
                    reg for reg in registros 
                    if str(reg.get('id', '')) in ids_set
                ]
                
                tempo_total = time.time() - inicio
                print(f"Bulk: {len(registros_filtrados)} registros em {tempo_total:.1f}s")
                
                return json_normalize(registros_filtrados)
        except Exception as e:
            print(f"Erro bulk: {e}")
        
        return json_normalize([])

    def executar_lotes():
        batch_size = 200
        lotes = [ids_unicos[i:i + batch_size] for i in range(0, len(ids_unicos), batch_size)]
        
        session = requests.Session()
        session.headers.update({
            'Content-Type': 'application/json',
            'Authorization': api_key,
            'ixcsoft': 'listar',
            'Connection': 'keep-alive'
        })
        session.mount('https://', requests.adapters.HTTPAdapter(
            pool_connections=20,
            pool_maxsize=40
        ))
        
        def processar_lote(batch_info):
            lote_num, ids_lote = batch_info
            ids_str = ','.join(map(str, ids_lote))
            url = f'{api_url}/{rota}'
            payload = json.dumps({
                'qtype': tabela,
                'query': f'({ids_str})',
                'oper': 'IN',
                'page': '1',
                'rp': '999999',
                'grid_param': f'{query}'
            })
            
            try:
                response = session.post(url, data=payload, timeout=45)
                if response.status_code == 200:
                    data = response.json()
                    return data.get("registros", [])
            except:
                pass
            return []
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            batch_infos = [(i, lote) for i, lote in enumerate(lotes)]
            futures = [executor.submit(processar_lote, batch_info) for batch_info in batch_infos]
            all_results = []
            
            for future in as_completed(futures):
                result = future.result()
                if result:
                    all_results.extend(result)
        
        session.close()
        
        tempo_total = time.time() - inicio
        print(f"Lotes: {len(all_results)} registros em {tempo_total:.1f}s")
        
        return json_normalize(all_results)
    
    # Lógica de decisão
    if estrategia == 'bulk':
        return executar_bulk()
    elif estrategia == 'lotes':
        return executar_lotes()
    else:
        total_registros = analisar_tabela()
        
        if total_registros:
            razao_solicitada = len(ids_unicos) / total_registros
            
            if razao_solicitada > 0.05 and total_registros < 200000:
                return executar_bulk()
            else:
                return executar_lotes()
        else:
            return executar_lotes()
        
# Criar fitlros adicionais mais fácil (query)
def grid_param(*filtros):
    
    filtros_strings = []
    
    for filtro in filtros:
        tb, op, valor, conector = filtro
        filtro_json = {
            "TB": tb,
            "OP": op,
            "P": valor,
            "C": conector,
            "G": tb
        }
        filtros_strings.append(json.dumps(filtro_json))
    
    resultado = "[" + ", ".join(filtros_strings) + "]"
    return resultado.replace('"', '\"')

# Substituir valores em branco/nulos
def substituir_nulos(valor, padrao):
    return padrao if isinstance(valor, str) and valor.strip() == '' else valor

In [96]:
cliente_contrato = requisicao_ixc("cliente_contrato", grid_param(("id_vd_contrato", "=", "58", "AND"), ("id_vd_contrato", "=", "93", "OR"), ("id_vd_contrato", "=", "52", "OR"), ("id_vd_contrato", "=", "94", "OR"), ("id_vd_contrato", "=", "60", "OR"), ("id_vd_contrato", "=", "4", "OR")))

In [97]:
cliente = requisicao_ixc_especifica("cliente", cliente_contrato['id_cliente'], "id")

Bulk: 31182 registros em 49.5s


In [98]:
cliente_contrato = cliente_contrato[['status', 'status_internet', 'id_cliente', 'id', 'id_vd_contrato', 'contrato']]
cliente_contrato = cliente_contrato.rename(columns={"id": "id_contrato"})

cliente = cliente[['id', 'telefone_comercial', 'telefone_celular', 'whatsapp', 'bairro', 'numero', 'cep', 'razao', 'data_cadastro']]
cliente = cliente.rename(columns={'id': 'id_cliente'})

df_alisson = pd.merge(cliente, cliente_contrato, on='id_cliente', how='left')

In [99]:
df_alisson

,id_cliente,telefone_comercial,telefone_celular,whatsapp,bairro,numero,cep,razao,data_cadastro,status,status_internet,id_contrato,id_vd_contrato,contrato
0,86857,,(11) 99025-4854,,Jardim Caiapia,850,06705-050,Carolina Alves Nunes,2025-07-26,A,A,125265,93,1 COMBO MULTI 30 Promocional
1,86811,,(11) 98425-7012,,Parque Rizzo II,3102,06702-300,Beatriz Silva de Sousa,2025-07-24,A,A,125204,60,2 COMBO MULTI 50 Promocional
2,86805,,(11) 96866-0146,,Vila Monte Serrat,146,06717-160,Francinete Pereira Santos,2025-07-24,A,A,125198,60,2 COMBO MULTI 50 Promocional
3,86800,,(11) 94963-2754,,Colinas de Cotia,230,06717-755,Kerlei Carlos de Oliveira,2025-07-24,A,A,125193,58,1 COMBO MULTI 50 Promocional
4,86720,,(11) 94329-8274,(11) 94329-8274,Ressaca,24,06887-400,Viviane de Novais Guimarães,2025-07-22,A,A,125095,60,2 COMBO MULTI 50 Promocional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37015,2,,(11) 99209-1378,(11) 98980-5374,Quinta dos Angicos,1153,06717-500,Antônio Carlos Silva Pereira,2011-11-25,I,D,48729,4,2 MULTI 50
37016,2,,(11) 99209-1378,(11) 98980-5374,Quinta dos Angicos,1153,06717-500,Antônio Carlos Silva Pereira,2011-11-25,I,D,14,4,2 MULTI 50
37017,2,,(11) 99209-1378,(11) 98980-5374,Quinta dos Angicos,1153,06717-500,Antônio Carlos Silva Pereira,2011-11-25,I,D,10,4,2 MULTI 50
37018,2,,(11) 99209-1378,(11) 98980-5374,Quinta dos Angicos,1153,06717-500,Antônio Carlos Silva Pereira,2011-11-25,I,D,9,4,2 MULTI 50


In [100]:
df_alisson.to_json('resultado_planosAntigos.json', orient='records', force_ascii=False, indent=2)
df_alisson.to_excel('resultado_planosAntigos.xlsx', index=False)

json_str = df_alisson.to_json(orient='records', force_ascii=False)
json_obj = json.loads(json_str)